In [ ]:
# !pip install -q six numpy scipy matplotlib scikit-image opencv-python imageio
# !pip install -q keras imgaug
# !pip install -q keras

In [ ]:
import keras
from imgaug import augmenters as ia

In [ ]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import SGD
from keras.callbacks import *
from keras.models import load_model

# from keras.callbacks import LearningRateScheduler,ModelCheckpoint,EarlyStopping,LambdaCallback
import os,sys,math

In [ ]:
import google
colab_dir='./'
file_name='EIP_CIFAR_10'
if hasattr(google,'colab'):
    from google.colab import drive
    drive.mount('/content/gdrive')
    colab_dir='/content/gdrive/My Drive/Colab Notebooks/'
model_file=colab_dir+file_name+'.h5'

In [ ]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [ ]:
#Augmentation and resizing
def augment(dataset,flip=0.65,blur=2.0,crop=(0,10)):
    seq = ia.Sequential([ia.Fliplr(flip),ia.GaussianBlur(sigma=(0, blur)),ia.Crop(px=crop)])
    return seq.augment_images(dataset)

def augmenter(X,y,start=.25,end=.75):
    ln=len(X)
    print('Before augmentation:',X.shape,y.shape)
    start=int(start*ln)
    end=int(end*ln)
    new_X=augment(X)[start:end]
    new_y=y[start:end]
    X=np.concatenate((X,new_X))
    y=np.concatenate((y,new_y))
    print('After augmentation:',X.shape,y.shape)
    return (X,y)

def resize_imgs(imgs,shape=(26,26)):
    seq = ia.Sequential([ia.Scale({"height": shape[0], "width": shape[1]})])
    return seq.augment_images(imgs)

In [ ]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2,num_layers=12):
    global compression
    temp = input
    for _ in range(num_layers):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])        
        temp = concat        
    return temp

def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
        Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)    
    return avg

def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)    
    return output
  
def dense_unit(input_layer,num_filter, dropout_rate,num_layers):
    dense_block=add_denseblock(input_layer, num_filter, dropout_rate,num_layers)
    transition_block=add_transition(dense_block, num_filter, dropout_rate)
    return transition_block
  
def dense_units_chain(n,input_layer,num_filter, dropout_rate,num_layers):
    dense_unit_=input_layer
    for i in range(n):
        dense_unit_=dense_unit(dense_unit_,num_filter, dropout_rate,num_layers)
    return dense_unit_

In [ ]:
#training time ops
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))

def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,math.floor((1+epoch)/epochs_drop))
    return lrate

class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,new_step_size=None):
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1       
        K.set_value(self.model.optimizer.lr, self.clr())

In [ ]:
# Load CIFAR10 Data
def load_data(resize=False,shape=(26,26)):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    if resize:
        x_train=resize_imgs(x_train,shape)
        x_test=resize_imgs(x_test,shape)
    
    (x_train, y_train) = augmenter(x_train, y_train,end=1)
    (x_test, y_test) = augmenter(x_test, y_test,end=1)
    return (x_train, y_train,x_test, y_test)

def create_model(input_shape):
    print('Creating model with input shape',input_shape)
    input = Input(input_shape)
    First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)
    hidden_dense_blocks = dense_units_chain(n_dense_blocks,First_Conv2D,num_filter,dropout_rate,num_layers)
    Last_Block = add_denseblock(hidden_dense_blocks, num_filter, dropout_rate)
    output = output_layer(Last_Block)
    model = Model(inputs=[input], outputs=[output])
    return model

In [ ]:
print('====================HYPER PARAMETERS====================')

In [ ]:
# Hyperparameters
batch_size = 100
num_classes = 10
max_epochs = 250
model_small_epochs=1
model_large_epochs=1

#num_layers = 40
num_layers = 24
num_filter = 12
compression = 0.5
dropout_rate = 0.2
n_dense_blocks = 3
smaller_input=(26,26,3)

In [ ]:
print('====================BEGIN OF SMALLER MODEL====================')

In [ ]:
x_train, y_train, x_test, y_test = load_data(resize=True,shape=smaller_input[:-1])
model_small = create_model(x_train.shape[1:])

#callbacks
model_checkpointer=ModelCheckpoint('small_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc',
                verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=5)
# lrate = LearningRateScheduler(step_decay)
# early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
clr = CyclicLR(base_lr=0.1, max_lr=0.2,step_size=8*(len(y_train)/batch_size))
callbacks = [clr, model_checkpointer]


model_small.compile(loss='categorical_crossentropy',metrics=['accuracy'],
              optimizer=SGD(lr=0.1, decay=1e-6, momentum=0.8, nesterov=True))

In [ ]:
model_small.summary()
try:
    model_small.load(model_file)
    print('Loaded model from file',model_file)
except Exception as e:
    print('Could not load any model from file')

In [ ]:
model_small.fit(x_train, y_train, batch_size=batch_size,
                    epochs=model_small_epochs,verbose=1, callbacks=callbacks,
                    validation_data=(x_test, y_test))
model_small.save(model_file)
print('Saved model_small to disk')

In [ ]:
print('====================END OF SMALLER MODEL====================')

In [ ]:
print('====================BEGIN OF LARGER MODEL====================')

In [ ]:
# TRAIN LARGER MODEL
x_train, y_train, x_test, y_test = load_data(resize=False)

#callbacks
model_checkpointer=ModelCheckpoint('large_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc',
                verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=5)
# lrate = LearningRateScheduler(step_decay)
# early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
clr = CyclicLR(base_lr=0.1, max_lr=0.2,step_size=8*(len(y_train)/batch_size))
callbacks = [clr, model_checkpointer]
model_large = create_model(x_train.shape[1:])

In [ ]:
print(os.popen('ls -lia '+colab_dir).read())

In [ ]:
try:
    model_prev = load_model(model_file)
    print('Loaded model from file',model_file)
except Exception as e:
    print(e, 'at line ', sys.exc_info()[2].tb_lineno)
    print('Attempting to load in memory, small model')
    model_prev=model_small

In [ ]:
model_prev.input_shape

In [ ]:
if model_prev.input_shape[1:]==smaller_input:
    print('Popping input layer')
    model_prev.layers.pop(0)
model_large.set_weights(model_prev.get_weights())
model_large.compile(loss='categorical_crossentropy',metrics=['accuracy'],
                    optimizer=SGD(lr=0.1, decay=1e-6, momentum=0.8, nesterov=True))
model_large.fit(x_train, y_train, batch_size=batch_size,
                epochs=model_lar_epochs,verbose=1, callbacks=callbacks,
                validation_data=(x_test, y_test))
model_large.save(model_file)
print("Saved model to disk")

In [ ]:
# Test the model
score = model_large.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
print('====================END OF LARGER MODEL====================')